# Make Pan-STARRS1 cutout images

- Inputs: list of visible comts, downloaded fits files
- Outputs: cutout images, statistics of observations (N, filters, etc.)

In [ ]:
import astropy
print(astropy.__version__)

import pandas as pd
import os
import requests
import numpy as np
print(np.__version__)
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle
import math

from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.visualization import simple_norm
from astroquery.jplhorizons import Horizons


In [ ]:
  def read_fits_data_header(path, ext=1):
        """
        Fallback: astropy.getdata/getheader with memmap disabled.
        """
        data = fits.getdata(path, ext=ext, memmap=False)
        header = fits.getheader(path, ext=ext)
        data = np.array(data, copy=True).astype(float)
        blank = header.get('BLANK', None)
        if blank is not None:
            data[data == blank] = np.nan
        return data, header

In [ ]:

def save_cutouts(fi_dir, eph_file, outdir, obj_jpl, n_per_page=32, fov_arcsec=20):
    df = pd.read_csv(eph_file)
    col_inst = "Telescope/Instrument"
    inst = "Pan-STARRS1"
    df = df[df[col_inst] == inst]

    col_ra, col_dec = "Object_RA", "Object_Dec"
    col_img = "Image"
    col_exptime = "Exptime"
    col_m1 = "M1"
    col_m2 = "M2"
    # Not fits header
    col_mjd = "MJD"

    # Short name for filename
    obj = obj_jpl.replace(" ", "")
    obj = obj.replace("/", "")

    cmap_map = {'g': 'Greens_r', 'r': 'Reds_r', 'i': 'gray', 'z': 'Purples_r', 'y': 'cividis'}
    default_cmap = 'gray'
    bbox = dict(boxstyle="round,pad=0.2", fc="black", alpha=0.5)

    os.makedirs(outdir, exist_ok=True)
    total = len(df)
    pages = math.ceil(total / n_per_page)
    

    fs_text = 10

    for page in range(pages):
        start_idx = page * n_per_page
        end_idx = min(start_idx + n_per_page, total)
        sub_df = df.iloc[start_idx:end_idx]

        n_plots = len(sub_df)
        n_cols = 8
        n_rows = 4
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(3*n_cols, 3*n_rows))
        axes = axes.flatten()

        for idx, row in enumerate(sub_df.itertuples()):
            ax = axes[idx]
            ra = getattr(row, col_ra)
            dec = getattr(row, col_dec)
            mjd = getattr(row, col_mjd)

  

            image_name = getattr(row, col_img)
            fits_path = os.path.join(fi_dir, image_name) + ".fits"

            if not os.path.exists(fits_path):
                ax.text(0.5, 0.5, 'File not found', ha='center', va='center', fontsize=12)
                ax.axis('off')
                continue

            try:
                data, header = read_fits_data_header(fits_path, ext=1)
            except Exception as e:
                ax.text(0.5, 0.5, f"Error reading FITS\n{e}", ha='center', va='center', fontsize=10)
                ax.axis('off')
                continue

            try:
                wcs = WCS(header)
            except Exception:
                wcs = None

            # Header
            mjd = header.get('MJD-OBS')

            jd  = mjd + 2400000.5
            # Calculate position with jpl horizons
            loc_PS1 = "F51"
            obj_H = Horizons(id=obj_jpl, location=loc_PS1, epochs=jd)
            eph = obj_H.ephemerides(extra_precision=True)
            ra_jpl, dec_jpl = eph['RA'][0], eph['DEC'][0]
            print(f"(mjd, jd) = ({mjd}, {jd})")
            print(f"  default (RA, Dec) = ({ra}, {dec})")
            print(f"  JPL     (RA, Dec) = ({ra_jpl}, {dec_jpl})")


            # Use jpl
            ra, dec = ra_jpl, dec_jpl
            
            cdelt1 = header.get('CDELT1', 0.258)
            pixscale = abs(cdelt1) * 3600.0  # arcsec/pix
            seeing_pix = header.get("HIERARCH CHIP.SEEING", None)
            filt_hdr = header.get("HIERARCH FPA.FILTER", None)
            band = None
            if isinstance(filt_hdr, str) and filt_hdr.strip():
                for ch in filt_hdr.strip():
                    if ch.isalpha():
                        band = ch.lower()
                        break
            cmap_name = cmap_map.get(band, default_cmap)

            out_of_fov = False
            try:
                x, y = wcs.wcs_world2pix(ra, dec, 0)
            except Exception:
                x, y = -1, -1

            if (x < 0) or (y < 0) or (x >= data.shape[1]) or (y >= data.shape[0]):
                out_of_fov = True

            if out_of_fov:
                ax.text(0.5, 0.5, 'OUT OF FOV', ha='center', va='center', fontsize=14, color='red')
                ax.axis('off')
            else:
                xi = float(np.round(x))
                yi = float(np.round(y))
                side_pix = fov_arcsec / pixscale             # arcsec → pix
                half = side_pix / 2.0
                y_min = int(max(yi - half, 0))
                y_max = int(min(yi + half, data.shape[0]))
                x_min = int(max(xi - half, 0))
                x_max = int(min(xi + half, data.shape[1]))
                cutout = data[y_min:y_max, x_min:x_max]

                try:
                    norm = simple_norm(cutout, 'sqrt', percent=99)
                except Exception:
                    norm = None

                ax.imshow(cutout, origin='lower', cmap=cmap_name, norm=norm)
                ax.axis('off')
                ax.set_title(image_name, fontsize=8) 

                # predicted position circle
                x_cut = x - x_min
                y_cut = y - y_min
                radius = 4
                if seeing_pix is not None and seeing_pix > 0:
                    radius = max(2, min(seeing_pix/2.0, 8))
                circ = Circle((x_cut, y_cut), radius=radius, edgecolor='yellow', facecolor='none', linewidth=1.5)
                ax.add_patch(circ)

                # scale bar (10 arcsec)
                bar_length_pix = 10.0 / pixscale  # arcsec / (arcsec/pix)
                bar_x = 5
                bar_y = 10
                ax.add_patch(Rectangle((bar_x, bar_y), bar_length_pix, 1, color='white'))
                ax.text(bar_x + bar_length_pix/2, bar_y + 2, "10\"", color='white', ha='center', va='bottom', fontsize=fs_text)

            # annotate header/df info using bbox
            exptime = getattr(row, col_exptime, None)
            if band is not None:
                ax.text(0.02, 0.98, f"Filter: {band}\nt_exp={exptime} s", color='w', fontsize=fs_text,
                        ha='left', va='top', transform=ax.transAxes, bbox=bbox)
            if seeing_pix is not None:
                seeing_arc = seeing_pix * pixscale
                ax.text(0.98, 0.98, f"Seeing: {seeing_pix:.2f}px / {seeing_arc:.2f}\"", color='w', fontsize=fs_text,
                        ha='right', va='top', transform=ax.transAxes, bbox=bbox)


            m1 = getattr(row, col_m1, None)
            m2 = getattr(row, col_m2, None)
            # Brightness
            Tmag = getattr(row, "Tmag", None)
            Nmag = getattr(row, "Nmag", None)

            if (m1 is not None) and (m2 is not None):
                ax.text(0.02, 0.08, f"(M1, M2) = ({m1}, {m2})", color='w', fontsize=fs_text,
                        ha='left', va='bottom', transform=ax.transAxes, bbox=bbox)
            if (Tmag is not None) and (Nmag is not None):
                ax.text(0.98, 0.02, f"(Tmag, Nmag) = ({Tmag}, {Nmag})", color='w', fontsize=fs_text,
                        ha='right', va='bottom', transform=ax.transAxes, bbox=bbox)

        # hide unused axes
        for ax in axes[n_plots:]:
            ax.axis('off')

        plt.tight_layout()
        print(obj)
        outfile = os.path.join(outdir, f"{obj}_{page+1}.jpg")
        plt.savefig(outfile, dpi=150)
        plt.close(fig)
        print(f"Saved {outfile}")


In [ ]:
# C2008 FK75
fi_dir = "data/C2008FK75/Pan-STARRS1"
eph_file = "data/visible2/C_2008FK75_visible.csv"

outdir = "fig"
obj_jpl = "C/2008 FK75"
fov_arcsec = 60
save_cutouts(fi_dir, eph_file, outdir, obj_jpl, fov_arcsec=fov_arcsec)

In [ ]:
# C2009 U5
fi_dir = "data/C2009U5/Pan-STARRS1"
eph_file = "data/visible2/C_2009U5_visible.csv"


outdir = "fig"
obj_jpl = "C/2009 U5"
fov_arcsec = 60
save_cutouts(fi_dir, eph_file, outdir, obj_jpl, fov_arcsec=fov_arcsec)

In [ ]:
# C2009F2
fi_dir = "data/C2009F2/Pan-STARRS1"
eph_file = "data/visible2/C_2009F2_visible.csv"

outdir = "fig"
obj_jpl = "C/2009 F2"
fov_arcsec = 60
save_cutouts(fi_dir, eph_file, outdir, obj_jpl, fov_arcsec=fov_arcsec)